In [1]:
%tensorflow_version 1.x
%cd /content
!git clone https://github.com/tomyaacov/RPNIComposition
%cd RPNIComposition
!pip install numpy==1.19.5

TensorFlow 1.x selected.
/content
fatal: destination path 'RPNIComposition' already exists and is not an empty directory.
/content/RPNIComposition
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
print(np.__version__)
import time
import sys
import random
random.seed(1)

1.19.5


In [3]:
data = []
with open("data/" + "accepted_words.csv", 'r') as f:
    for line in f:
        data.append(line.strip().split(",") + ["1"])
with open("data/" + "rejected_words.csv", 'r') as f:
    for line in f:
        data.append(line.strip().split(",") + ["0"])
random.shuffle(data)
train = data[:int(len(data) * 0.8)]
test = data[int(len(data) * 0.8):]

In [4]:
train_data = [x[:-1] for x in train]
test_data = [x[:-1] for x in test]
train_labels = np.array([int(x[-1]) for x in train])
test_labels = np.array([int(x[-1]) for x in test])

In [5]:
def get_enc(c):
  if c[0] == "X":
    return int(c[1:])
  else:
    return 9+int(c[1:])
train_data_enc = [[get_enc(c) for c in d] for d in train_data]
test_data_enc = [[get_enc(c) for c in d] for d in test_data]


In [6]:
max_length = 9
train_data_pad = pad_sequences(train_data_enc,maxlen=max_length,padding='post',value=18)
test_data_pad = pad_sequences(test_data_enc,maxlen=max_length,padding='post',value=18)

In [7]:
model=Sequential()

model.add(Embedding(19,10,input_length=9))
model.add(LSTM(32,input_shape=(9, 10),activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(32,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 9, 10)             190       
_________________________________________________________________
lstm (LSTM)                  (None, 9, 32)             5504      
_________________________________________________________________
dropout (Dropout)            (None, 9, 32)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)        

In [10]:
start_time = time.time()
history = model.fit(train_data_pad, train_labels, epochs=10, batch_size=64, validation_data=(test_data_pad, test_labels))
print("lstm:")
print("--- %s seconds training ---" % (time.time() - start_time))


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 464us/sample - loss: 0.4615 - acc: 0.7970 - val_loss: 0.5165 - val_acc: 0.7610
Epoch 2/10
1000/1000 [==============================] - 0s 485us/sample - loss: 0.4580 - acc: 0.7970 - val_loss: 0.5187 - val_acc: 0.7610
Epoch 3/10
1000/1000 [==============================] - 0s 479us/sample - loss: 0.4495 - acc: 0.7970 - val_loss: 0.5072 - val_acc: 0.7610
Epoch 4/10
1000/1000 [==============================] - 0s 488us/sample - loss: 0.4373 - acc: 0.7970 - val_loss: 0.4998 - val_acc: 0.7610
Epoch 5/10
1000/1000 [==============================] - 0s 492us/sample - loss: 0.4293 - acc: 0.7970 - val_loss: 0.5000 - val_acc: 0.7610
Epoch 6/10
1000/1000 [==============================] - 0s 479us/sample - loss: 0.4109 - acc: 0.7970 - val_loss: 0.4615 - val_acc: 0.7610
Epoch 7/10
1000/1000 [==============================] - 0s 469us/sample - loss: 0.3977 - acc: 0.7970 - val_loss: 0.4436 - va

In [11]:
print("lstm acc", history.history["acc"])
lstm_acc = history.history["acc"]
print("lstm val acc", history.history["val_acc"])
lstm_val_acc = history.history["val_acc"]

lstm acc [0.797, 0.797, 0.797, 0.797, 0.797, 0.797, 0.797, 0.797, 0.797, 0.797]
lstm val acc [0.761, 0.761, 0.761, 0.761, 0.761, 0.761, 0.761, 0.761, 0.761, 0.761]


In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
path = '/content/drive/MyDrive/research/tic_tac_toe_model_history.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df = pd.DataFrame.from_dict(history.history)
  df.to_csv(f)

path = '/content/drive/MyDrive/research/tic_tac_toe_model.h5'
model.save(path)